# Laboratoire 2
## Gyroscopes à taux (rate gyroscope)

**Matériel nécessaire fourni** : table tournante, surface antidérapante,
règle de 30 cm, masking tape, LEGOs inclinés).

Ce laboratoire utilise le gyro inclus dans le système Kobuki. Il s'agit du STMicroelectronics [L3G4200D](http://www.st.com/content/ccc/resource/technical/document/datasheet/04/46/d6/00/be/d9/46/ae/CD00265057.pdf/files/CD00265057.pdf/jcr:content/translations/en.CD00265057.pdf).

![img](img/gyro_schema.png)
![img](img/gyro_datasheet.png)